In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [ ]:
data =pd.read_csv('/kaggle/input/stockheadings/Data.csv', encoding="ISO-8859-1")

In [ ]:
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [ ]:
nltk.download('wordnet')

In [ ]:
train = data[data['Date'] < '20150101']
test = data[data['Date'] > '20141231']

In [ ]:
x_train=train.iloc[:,2:27]
y_train=train.iloc[:,1:2]

x_test=test.iloc[:, 2:27]
y_test=test.iloc[:,1:2]

In [ ]:
lem=WordNetLemmatizer()

In [ ]:
def process(x):
    x['headings']=x.apply(lambda x: ' '.join(map(str,x)), axis=1)
    x=x['headings']
    for i in range(len(x)):
        words=re.sub('[^a-zA-Z]', ' ', x.iloc[i])
        words=nltk.word_tokenize(words)
        words=[lem.lemmatize(word.lower()) for word in words if word not in stopwords.words('english')]
        x.iloc[i]=words
    return x

In [ ]:
x_train=process(x_train)

In [ ]:
full_data=data.iloc[:,2:]

In [ ]:
full_data=process(full_data)

In [ ]:
x_test=process(x_test)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model=Word2Vec(full_data,window=10, min_count=2)

In [ ]:
vocab=model.wv.index_to_key
vocab

In [ ]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [ ]:
X=[]

In [ ]:
for i in range(len(x_train)):
    X.append(avg_word2vec(x_train[i]))

In [ ]:
test=[]

In [ ]:
for i in x_test:
    test.append(avg_word2vec(i))

In [ ]:
len(test)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
class_model=XGBClassifier()

In [ ]:
class_model.fit(X, y_train)

In [ ]:
y_pred=class_model.predict(test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))